In [2]:
import numpy as np
import pandas as pd 
import os
import numpy_financial as npf

In [3]:
# DOWNLOAD DATA
os.chdir("G:\My Drive\שנה ב\שנה ב סמסטר ב\פרויקט דאטא\Analytics-Based Project_\y2019")
Q1_19 = pd.read_csv("Q1_2019.csv",low_memory=False)
Q2_19 = pd.read_csv("Q2_2019.csv",low_memory=False)
Q3_19 = pd.read_csv("Q3_2019.csv",low_memory=False)
Q4_19 = pd.read_csv("Q4_2019.csv",low_memory=False)

In [4]:
# DOWNLOAD DATA
os.chdir("G:\My Drive\שנה ב\שנה ב סמסטר ב\פרויקט דאטא\Analytics-Based Project_\y2019")
Q1_18 = pd.read_csv("Q1_2018.csv",low_memory=False)
Q2_18 = pd.read_csv("Q2_2018.csv",low_memory=False)
Q3_18 = pd.read_csv("Q3_2018.csv",low_memory=False)
Q4_18 = pd.read_csv("Q4_2018.csv",low_memory=False)

In [5]:
# first check shapes
print(Q1_19.shape,Q1_18.shape)
print(Q2_19.shape,Q2_18.shape)
print(Q3_19.shape,Q3_18.shape)
print(Q4_19.shape,Q4_18.shape)

(133887, 151) (133887, 152)
(97854, 151) (97854, 152)
(99120, 151) (99120, 152)
(103546, 151) (103546, 152)


In [6]:
# making 2 big Date Frames
all_year_19 = pd.concat([Q1_19,Q2_19,Q3_19,Q4_19])
all_year_18 = pd.concat([Q1_18,Q2_18,Q3_18,Q4_18])

In [7]:
# have a look
all_year_19.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,payment_lifetime
0,76143686,NaN,9000,9000,9000.0,36 months,15.31%,313.36,C,C5,...,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,365
1,76132182,NaN,24000,24000,24000.0,60 months,6.49%,469.48,A,A2,...,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,1156
2,76143234,NaN,8000,8000,8000.0,36 months,10.75%,260.97,B,B4,...,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,1095
3,75710736,NaN,12800,12800,12800.0,36 months,11.99%,425.09,C,C1,...,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,1126
4,75858516,NaN,5000,5000,5000.0,36 months,16.29%,176.51,D,D1,...,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,761


In [12]:
# now lets take only relevant loans

# including all closed (also 60 months)
# all_year_19 = all_year_19[all_year_19.loan_status.isin(["Fully Paid", "Charged Off", "Default"])]

# including only 36 months term loans
all_year_19.term.describe()
#all_year_19 = all_year_19[all_year_19.term == " 36 months"]

count         323495
unique             1
top        36 months
freq          323495
Name: term, dtype: object

In [ ]:
all_year_19.payment_lifetime.describe()
# for low values appiring by missing date (start or finish) 
# i assing max time (36 months ) for all negative values:
## it will only hurt (lower) relised return as 
## we asuming those payments are recived as late as possible
### values with 0 get 1 (its best to check in 18 tabbles)
months_vector = list(round(all_year_19.payment_lifetime/30.4))
for i in range(len(months_vector)) : 
    if months_vector[i] < 0 :
        months_vector[i] = 36
    # ! problem with 0 values ! change this ! 
    elif months_vector[i] == 0 : 
        months_vector[i] = 1
    # ! problem with 0 values ! change this ! 

In [ ]:
# now lets take another look
sum(np.array(months_vector) <= 0 )
all_year_19["months"] = months_vector
all_year_19.months.describe()

In [ ]:
# lets calculate the monthly payments 
all_year_19["monthly_paymnent"] = all_year_19["total_pymnt"] / all_year_19["months"]

In [ ]:
all_year_19.loan_status.unique()

In [ ]:
# now we can make shpithzer Monthly payments 
shpizer_payments = []

for _, row in all_year_19.iterrows():
    pymnts = [row['funded_amnt']*-1]  
    pymnts += [row['monthly_paymnent']] * int(row['months'])  
    shpizer_payments.append(pymnts) 

In [ ]:
# Monthly rate !
realised_rate = []
for i in shpizer_payments : 
    realised_rate.append(npf.irr(i))
# let's turn it to yearly
yearly_realised_rates = ((np.array(realised_rate) + 1)**12) -1 

In [ ]:
yearly_realised_rates

In [ ]:
# now we add that list to our Data Frame as relised rtrn
all_year_19["year_realised_return"] = yearly_realised_rates

In [ ]:
all_year_19.year_realised_return.describe()

In [ ]:
grade_groups_year_19 = all_year_19.groupby(["grade"])
grade_groups_year_19.mean().year_realised_return

In [ ]:
grade_groups_year_19.std().year_realised_return

In [ ]:
grade_groups_year_19 = all_year_19.groupby(["grade","sub_grade"])
grade_groups_year_19.mean().year_realised_return

In [ ]:
grade_groups_year_19 = all_year_19.groupby(["grade","sub_grade"])
grade_groups_year_19.std().year_realised_return

In [ ]:
# cheack 1 - lets make a worst case senario in which the payment is fully paid on the last month
worst_rlsd_rate = []
shpizer_payments_2 = []

for _, row in all_year_19.iterrows():
    pymnts = [row['funded_amnt']*-1]  
    pymnts += [0] * (int(row['months'])-1) 
    pymnts += [row['total_pymnt']]
    shpizer_payments_2.append(pymnts) 

In [ ]:
# here are the worst possible payments according to dates
shpizer_payments_2[0]

In [ ]:
# now agian using irr
# Monthly rate to yearly rate
worst_realised_rate = []
for i in shpizer_payments_2 :
    # Monthly rate to yearly rate
    worst_realised_rate.append(((npf.irr(i)+1)**12)-1)

In [ ]:
all_year_19["worst_realised_rate"] = worst_realised_rate 

In [ ]:
all_year_19.worst_realised_rate.describe()

In [ ]:
grade_groups_year_19 = all_year_19.groupby(["grade","sub_grade"])
grade_groups_year_19.mean().worst_realised_rate

In [ ]:
## now lets see what makes some loans have over 100% realised int rate
check1 = all_year_19[["funded_amnt","total_pymnt","months","monthly_paymnent","year_realised_return","int_rate","worst_realised_rate"]][all_year_19.year_realised_return >= 1]

In [ ]:
# check1.year_realised_return.describe()
check1.year_realised_return.sort_values(ascending=False)

In [ ]:
# only 8 loans 
check1 

In [ ]:
# thats wired... lets have a closer look 
((npf.irr([-20000,5881,5881,5881,5881])+1)**12)-1
((npf.irr([-2000,2166])+1)**12)-1
# ok, lets check how they have low months
check2 = all_year_19[["issue_d","last_pymnt_d","months","funded_amnt","total_pymnt","year_realised_return"]][all_year_19.year_realised_return >= 1]

In [ ]:
check2
# they acually look valid... :( 
# we can again think of if irr asumption (reinvesting) is the best fit for us....
# maybe if we set an YTM% of 2% as the reinvesment rate in the IRR ?..?..?..?..? 

In [ ]:
# Assuming reinvestment at a specific rate (2% yearly)
reinvestment_rate = 0.02

# Monthly rate!
realized_rate = []
for i in shpizer_payments:
    realized_rate.append(npf.mirr(i, reinvestment_rate, reinvestment_rate/12))

# Calculate realized rates with specified reinvestment rate
yearly_realized_rates = ((np.array(realized_rate) + 1) ** 12) - 1

In [ ]:
len(all_year_19.columns),len(all_year_18.columns)

In [ ]:
columns_19 = list(all_year_19.columns)

In [ ]:
# these are all the 18 features that are not included in 19
exclude_18 = all_year_18.columns[all_year_18.columns.isin(all_year_19)== False ]
exclude_18

In [ ]:
# these are all the 19 features that are not included in 18
exclude_19 = all_year_19.columns[all_year_19.columns.isin(all_year_18)== False ]
exclude_19

In [ ]:
# we want to check for perminant data - all of this are not ! 
all_year_19_copy = all_year_19.drop(columns=exclude_19)
all_year_18_copy = all_year_18.drop(columns=exclude_18)

ValueError: Can only compare identically-labeled DataFrame objects